### **Azure OpenAI Function Calling**

In [1]:
from openai import AzureOpenAI
import json
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('default', 'AZURE_OPENAI_KEY')
api_endpoint = config.get('default','AZURE_OPENAI_ENDPOINT')

In [3]:
client = AzureOpenAI(
  azure_endpoint = api_endpoint, 
  api_key=api_key,  
  api_version="2024-02-15-preview"
)

In [4]:

def get_current_stock_price(symbol):
    """Get the current stock price for a given ticker(symbol)"""
    if "msft" in symbol.lower():
        return json.dumps({"symbol": "MSFT", "price": "23"})
    elif "aapl" in symbol.lower():
        return json.dumps({"symbol": "AAPL", "price":  "72"})
    elif "amzn" in symbol.lower():
        return json.dumps({"symbol": "AMZN", "price":  "22"})
    else:
        return json.dumps({"symbol": symbol, "price": "unknown"})

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_stock_price",
            "description": "Get the current stock price",
            "parameters": {
                "type": "object",
                "properties": {
                "symbol": {
                    "type": "string",
                    "description": "It's the symbol of a stock like MSFT"
                }
                },
                "required": [
                "symbol"
                ]
            }
        },
    }
]

In [6]:
messages = [{"role": "user", "content": "What's the current stock price for Apple?"}]

In [7]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)
response_message = response.choices[0].message

In [8]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qTNzrl14kisNroaa5MzfVZMY', function=Function(arguments='{\n  "symbol": "AAPL"\n}', name='get_current_stock_price'), type='function')])

In [9]:
tool_calls = response_message.tool_calls

In [10]:
tool_calls

[ChatCompletionMessageToolCall(id='call_qTNzrl14kisNroaa5MzfVZMY', function=Function(arguments='{\n  "symbol": "AAPL"\n}', name='get_current_stock_price'), type='function')]

In [11]:
available_functions = {
    "get_current_stock_price": get_current_stock_price,
} 

messages.append(response_message)  

In [12]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        symbol=function_args.get("symbol")
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )  
    print(messages)
second_response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
)  
print("Model Response:")
print(second_response.choices[0].message.content) 

[{'role': 'user', 'content': "What's the current stock price for Apple?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qTNzrl14kisNroaa5MzfVZMY', function=Function(arguments='{\n  "symbol": "AAPL"\n}', name='get_current_stock_price'), type='function')]), {'tool_call_id': 'call_qTNzrl14kisNroaa5MzfVZMY', 'role': 'tool', 'name': 'get_current_stock_price', 'content': '{"symbol": "San Francisco", "price": "72"}'}]
Model Response:
The current stock price for Apple is $72.
